In [1]:
import agent
from importlib import reload 
import cube_sim
# import eval
# reload(cube_sim)
# reload(agent)
import itertools
import numpy as np
import pandas as pd
from pathlib import Path
import os
import pickle
import json
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib widget

In [2]:
a = agent.ApproximateQAgent()

In [151]:
a = a.load(fname='12_05_v1_episode_2750',inpath=Path('./output'))

Loading agent from output


In [152]:
a.weights

{'b': 13.960930337445278,
 'one_color_face': 36.277555250603406,
 'two_color_face': 16.012270676163357,
 'three_color_face': 16.54629260148434,
 'four_color_face': 14.729463496415704,
 'front_top': -8.760342718519757,
 'front_bottom': -9.98672020076564,
 'front_left': -11.539384111592446,
 'front_right': -6.34403721584759,
 'front_back': -12.879418068106656,
 'top_bottom': -12.261890114390152,
 'top_left': -7.334959430516729,
 'top_right': -13.024153532470814,
 'top_back': -7.993386308274473,
 'bottom_left': -8.357399733327433,
 'bottom_right': -10.574850588443011,
 'bottom_back': -11.609085455490579,
 'left_right': -10.664077423526702,
 'left_back': -9.241969999524391,
 'right_back': -6.071880720951559,
 'full_layers': 28.96119334625492}

In [153]:
epdf = pd.DataFrame.from_dict(a.metadata,orient='index')

In [154]:
epdf.head()

,MovesToGoal,EpisodeRewards,Weights,TotalRunTime,EpisodeRunTime,EndEpsilon
ep_1,999,4608.0,"{'b': 13.960930337445278, 'one_color_face': 36...",6.112919,6.112919,0.899810
ep_2,999,5148.0,"{'b': 13.960930337445278, 'one_color_face': 36...",12.180526,6.067607,0.899620
ep_3,999,4836.0,"{'b': 13.960930337445278, 'one_color_face': 36...",18.200423,6.019897,0.899430
ep_4,999,5397.0,"{'b': 13.960930337445278, 'one_color_face': 36...",24.269508,6.069084,0.899239
ep_5,999,5210.0,"{'b': 13.960930337445278, 'one_color_face': 36...",30.377366,6.107858,0.899048


In [155]:
solved

,EpisodeRewards
ep_318,5592.73
ep_480,6102.88
ep_529,6197.65
ep_639,6524.71
ep_645,6496.71
...,...
ep_2277,11838.34
ep_2288,11719.39
ep_2347,12098.63
ep_2445,11242.61


In [157]:
plt.close("all")
x = epdf.index

rwdcolor = 'tab:blue'
fig, ax = plt.subplots(figsize=(8,5))
ax.plot(x,epdf[['EpisodeRewards']].rolling(100).mean(), color=rwdcolor,label='Rewards')
ax.tick_params(axis='x',rotation=60)
ax.tick_params(axis='y', labelcolor=rwdcolor)
ax.set_xlim(left=80)
# ax.set_xlabel('Episode')  # Add an x-label to the axes.
ax.set_ylabel('Rewards', color=rwdcolor)  # Add a y-label to the axes.
ax.set_title("Rewards Per Episode (Rolling Mean of Previous 100 Episodes)")  # Add a title to the axes.
xtick = list(epdf.index[99::100])
xtick.append(epdf.index[-1])
xlab = [f'Ep {i[3:]}' for i in xtick]
ax.set_xticks([])
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
# ax.text(50,8000, size = 10,
#         s='''If Goal State is True:
#     Reward = 1000
# Else:
#     Reward = (# Full Layers)*10 + (# 1 Color Faces)*6 +
#     (# 2 Color Faces)*3 + # 3 Color Faces - living_tax''',
#         ha="left",
#         va="center",
#         bbox=dict(boxstyle="round",
#                    ec='k',
#                    fc='w',
#                    ))

ax2 = ax.twinx()  # instantiate a second axes that shares the same x-axis

epscolor = 'red'
ax2.set_ylabel('Epsilon Value', color=epscolor)  # we already handled the x-label with ax1
ax2.plot(x, epdf['EndEpsilon'], color=epscolor,label="Epsilon",lw=1,ls='--')
ax2.tick_params(axis='y', labelcolor=epscolor)
ax.set_xticks(xtick)
ax.set_xticklabels(xlab)
# ax2.spines['right'].set_visible(False)
ax2.spines['top'].set_visible(False)

solved = epdf[['EpisodeRewards']].rolling(100).mean()[epdf.MovesToGoal < 999]

ax.plot(solved, color='tab:blue',label="Solved",ls='',marker='o',markerfacecolor='white',ms=5)

fig.legend(loc=(.75,.375))

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
c = cube_sim.cube()
a.getFeatures(c)

{'full_layers': 6,
 'one_color_face': 6,
 'two_color_face': 0,
 'three_color_face': 0,
 'four_color_face': 0,
 'front_top': 0.5,
 'front_bottom': 0.5,
 'front_left': 0.5,
 'front_right': 0.5,
 'front_back': 0.5,
 'top_bottom': 0.5,
 'top_left': 0.5,
 'top_right': 0.5,
 'top_back': 0.5,
 'bottom_left': 0.5,
 'bottom_right': 0.5,
 'bottom_back': 0.5,
 'left_right': 0.5,
 'left_back': 0.5,
 'right_back': 0.5,
 'b': 1}

In [ ]:
# How does the agent perform over 1000 iterations with a 5000 move max count

In [8]:
e = agent.evaluator(a,numIterations=1000)

In [9]:
e.run()

Begin Evaluation
Starting iteration 0
Starting State:


[' ' ' '] ['y' 'o'] [' ' ' '] [' ' ' '] 
[' ' ' '] ['o' 'b'] [' ' ' '] [' ' ' '] 

['r' 'g'] ['w' 'w'] ['r' 'b'] ['y' 'b'] 
['g' 'o'] ['b' 'w'] ['r' 'o'] ['g' 'y'] 

[' ' ' '] ['w' 'g'] [' ' ' '] [' ' ' '] 
[' ' ' '] ['r' 'y'] [' ' ' '] [' ' ' '] 
On Move 0
Run time: 0.001994609832763672
Hit max moves: 5000
Total Run time: 15.05872654914856
2020-12-05 06:04:23.866668
Starting iteration 1
Starting State:


[' ' ' '] ['b' 'y'] [' ' ' '] [' ' ' '] 
[' ' ' '] ['r' 'o'] [' ' ' '] [' ' ' '] 

['r' 'w'] ['g' 'w'] ['b' 'o'] ['b' 'w'] 
['w' 'r'] ['b' 'y'] ['g' 'o'] ['g' 'o'] 

[' ' ' '] ['y' 'r'] [' ' ' '] [' ' ' '] 
[' ' ' '] ['g' 'y'] [' ' ' '] [' ' ' '] 
On Move 0
Run time: 0.000997304916381836
Hit max moves: 5000
Total Run time: 14.888214111328125
2020-12-05 06:04:38.757870
Starting iteration 2
Starting State:


[' ' ' '] ['b' 'y'] [' ' ' '] [' ' ' '] 
[' ' ' '] ['w' 'g'] [' ' ' '] [' ' ' '] 

['o' 'b'] ['o' 'y'] ['r' 'b'] ['r' 'y'] 
[

KeyboardInterrupt: 

In [ ]:
# How does a completely random agent perform

In [ ]:
a = agent.ApproximateQAgent()